# Gopigo navigation & SLAM

Using SLAM (short for Simultaneous Localization and Mapping) techniques, you will be able to execute autonomous navigation with GoPiGo3.

SLAM is a technique used in robotics to explore and map an unknown environment while estimating the pose of the robot itself. As it moves all around, it will be acquiring structured information of the surroundings by processing the raw data coming from its sensors.

For optimal and easy-to-understand coverage of the topic of SLAM, we will implement a 360º-coverage Laser Distance Sensor (LDS) in the virtual robot. 

There are low-cost versions of this sensor technology, such as EAI YDLIDAR X4 (available at https://www.robotshop.com/es/es/escaner-laser-360-ydlidar-x4.html), which is the one we will make use of in the next chapter.


### Install ROS navigation & SLAM packages
First, let's prepare your machine with the required ROS packages needed for the navigation stack (http://wiki.ros.org/navigation):

In [ ]:
sudo apt install ros-melodic-navigation

And finally the slam_gmapping package, that is already available in its binary version (https://wiki.ros.org/slam_gmapping)

In [ ]:
sudo apt-get install ros-melodic-slam-gmapping

Open the .bashrc file and verify the ROS_IP and ROS_MASTER_URI environment variables and source to the proper workspace:

source ~/rUBotCoop_LabProject/devel/setup.bash

export ROS_IP=192.168.18.83

export ROS_MASTER_URI=http://192.168.18.83:11311

## What do you need to perform robot navigation with ROS?
- Mapping: First, you need a map
- Localization: Next you need to localize the robot on that map
- Path Planning: Now you can send goal locations to the robot
- Obstacle avoidance: Finally, you need to avoid obstacles

### MAP generation
The very first thing you need in order to perform Navigation is... of course, a Map. You need a Map of the environment where you want your robot to navigate at. A Map is just a representation of an environment created from the sensor readings of the robot (for example, from the laser, among others). So just by moving the robot around the environment, you can create an awesome Map of it! In terms of ROS Navigation, this is knows as Mapping. 

### Localization the robot on that map
This means, in order to perform a proper Navigation, your robot needs to know in which position of the Map it is located and with which orientation (that is, which direction the robot is facing) at every moment. In terms of ROS Navigation, this is known as Localization. Let's see a quick demonstration of localization.

### Plan a goal location to the robot
For this, we'll need some kind of system which tells the robot WHERE to go, at first, and HOW to go there, at last. In ROS, we call this system the Path Planning. The Path Planning basically takes as input the current location of the robot and the position where the robot wants to go, and gives us as an output the best and fastest path in order to reach that point.

### Obstacle avoidance
Basically, the Obstacle Avoidance system breaks the big picture (Map) into smaller pieces, which updates in real-time using the data it's getting from the sensors. This way, it assures it won't be surprised for any sudden change in the environment, or by any obstacle that appears in the way. Use the 2D Nav Goal to send the robot to different places of the Caffe. Select a destination point that requires the robot to avoid the newly inserted object, so you can see how the path is being modified by the obstacle and how the robot is avoiding it and not colliding with it.

In order to have the ROS Navigation Stack working properly, you need to provide some data to the system , depending on which is the robot you want to use and how it is built. That is, you need to have your robot properly configured.

Robot Configuration is extremely important in all the navigation modules. For instance, in the Mapping system, if you don't tell the system WHERE does your robot have the laser mounted on, which is the laser's orientation, which is the position of the wheels in the robot, etc., it won't be able to create a good and accurate Map. And as you may already know at this point, if we don't have a good Map in ROS Navigation, we have NOTHING!



## Navigation Stack

The Navigation Stack is a set of ROS nodes and algorithms which are used to autonomously move a robot from one point to another, avoiding all obstacles the robot might find in its way. The ROS Navigation Stack comes with an implementation of several navigation related algorithms which can help you perform autonomous navigation in your mobile robots.

Following there is a figure with the basic building blocks of the Navigational stack taken from the ROS official website (http://wiki.ros.org/navigation/Tutorials/RobotSetup).

<img src="./Images/3_Nav_stack.png">

## The move_base node
This is the most important node of the Navigation Stack. It's where most of the "magic" happens.

The main function of the move_base node is to move a robot from its current position to a goal position with the help of other Navigation nodes. This node links the global planner and the local planner for the path planning, connecting to the rotate recovery package if the robot is stuck in some obstacle, and connecting global costmap and local costmap for getting the map of obstacles of the environment.

Following is the list of all the packages which are linked by the move_base node:

- global-planner
- local-planner
- rotate-recovery
- clear-costmap-recovery
- costmap-2D

Following are the other packages which are interfaced to the move_base node:

- map-server
- AMCL
- gmapping

<img src="./Images/3_Nav_stack2.png">

## SLAM gmapping package

Simultaneous Localization and Mapping (SLAM) allows the robot to build a map of the environment using the following two sources of information: 
- Robot pose estimation, coming from the internal odometry (rotary encoders) and IMU sensor data 
- Distance to objects, obstacles and walls, coming from distance sensors, the LDS in particular 

The gmapping ROS package is an implementation of a specific SLAM algorithm called gmapping. 

The gmapping package contains a ROS Node called slam_gmapping, which allows you to create a 2D map using the laser and pose data that your mobile robot is providing while moving around an environment. This node basically reads data from the laser and the transforms of the robot, and turns it into an occupancy grid map (OGM).

The generated map is published during the whole process into the /map topic, which is the reason you could see the process of building the map with Rviz (because Rviz just visualizes topics).

The /map topic uses a message type of nav_msgs/OccupancyGrid, since it is an OGM. 

## map_server package
Another of the packages available in the ROS Navigation Stack is the map_server package. This package provides the map_saver node, which allows us to access the map data from a ROS Service, and save it into a file.

Map information is stored using two files: 
- A .pgm format file, known as portable graymap format. 
- A .yaml file containing the configuration of the map. 

In [ ]:
image: ./test_map.pgm
resolution: 0.010000
origin: [-20.000000, -20.000000, 0.000000]
negate: 0
occupied_thresh: 0.65
free_thresh: 0.196

# Navigation with gopigo3 in a custom maze
The proposed navigation of gopigo3 robot in a custom maze has to be done following the next steps:

### 1. Spawn the robot in our custom made maze
We open a new terminal and type:


In [ ]:
roslaunch virtual_slam gopigo3_world_stl.launch

### 2. Generate the MAP
We will start the slam_gmapping node. This node is highly configurable and has lots of parameters you can change in order to improve the mapping performance. (http://wiki.ros.org/gmapping)

Let's now check some of the most important ones that usually have to be changed:

- base_frame (default: "base_link"): Indicates the name of the frame attached to the mobile base.
- map_update_interval (default: 5.0): Sets the time (in seconds) to wait until update the map.
- delta (default: 0.05): Sets the resolution of the map
- linearUpdate (default: 1.0): Sets the linear distance that the robot has to move in order to process a laser reading.
- temporalUpdate (default: -1.0): Sets the time (in seconds) to wait between laser readings. If this value is set to -1.0, then this function is turned off.

Open the "gmapping.launch" file and change properly the parameters you consider.

In [ ]:
roslaunch virtual_slam gopigo3_slam_stl.launch

Teleoperate the robot to make it cover as much as possible of the surface of the current Gazebo world. Let's do this as usual with the teleoperation package:

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel

Or use the navigation program you have designed to follow the walls for exemple to properly generate the map.

Once you've finished the exploration, save the map, generating two files of the formats indicated in the preceding SLAM process subsection, that is, .pgm and .yaml

<img src="./Images/3_slam_map.png">

In [ ]:
rosrun map_server map_saver -f ~/Escriptori/ROS/gopigo_pc_ws/src/virtual_slam/maps/maze1

You will get two files in the specified folder of your workspace: maze1.pgm and maze1.yaml.

Provided with the map, we are ready to perform robot navigation with the GoPiGo3.

### 3. Robot Navigation
When the robot moves around a map, it needs to know which is its POSE within the map.

The AMCL (Adaptive Monte Carlo Localization) package provides the amcl node, which uses the MCL system in order to track the localization of a robot moving in a 2D space. This node subscribes to the data of the laser, the laser-based map, and the transformations of the robot, and publishes its estimated position in the map. On startup, the amcl node initializes its particle filter according to the parameters provided.

This AMCL node is also highly customizable and we can configure many parameters in order to improve its performance. (http://wiki.ros.org/amcl)

Let's have a look at some of the most important ones:

General Parameters

- odom_model_type (default: "diff"): It puts the odometry model to use. It can be "diff," "omni," "diff-corrected," or "omni-corrected."
- base_frame_id (default: "base_link"): Indicates the frame associated with the robot base.

Filter Parameters

These parameters will allow you to configure the way that the particle filter performs.
- update_min_d (default: 0.2): Sets the linear distance (in meters) that the robot has to move in order to perform a filter update.
- update_min_a (default: π/6.0): Sets the angular distance (in radians) that the robot has to move in order to perform a filter update.

So, basically, we have to do the following:
- You have to close the slam_gmapping node
- you launched an amcl node using the preconfigured amcl_demo.launch file.
- you set up an initial pose by using the 2D Pose Estimate tool (which published that pose to the /initialpose topic).
- Then, you started moving the robot around the room, and the amcl node began reading the data published into the laser topic (/scan), the map topic(/map), and the transform topic (/tf), and published the estimated pose where the robot was in to the /amcl_pose and the /particlecloud topics.
- Finally, via RViz, you accesed the data being published by this node into the /particlecloud topic, so you were able to vizualize it, thanks to the cloud of "arrows," which were indicating the most probable position the robot was in, and its orientation.

In [ ]:
roslaunch virtual_slam gopigo3_navigation_stl.launch map_file:=$HOME/Escriptori/ROS/gopigo_pc_ws/maps/maze1.yaml

<img src="./Images/3_slam_stl2.png">